In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("scikit-learn-mooc/datasets/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.2, random_state=42
)

In [3]:
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1
)
preprocessor = make_column_transformer(
    (categorical_preprocessor, selector(dtype_include=object)),
    remainder="passthrough",
)

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", HistGradientBoostingClassifier(random_state=42)),
    ]
)

In [5]:
import numpy as np
from sklearn.model_selection import cross_val_score

learning_rates = [0.01, 0.1, 1, 10]
max_leaf_nodes_list = [3, 10, 30]

best_score = -np.inf
best_params = None

for lr in learning_rates:
    for mln in max_leaf_nodes_list:
        model.set_params(
            classifier__learning_rate=lr,
            classifier__max_leaf_nodes=mln
        )
        scores = cross_val_score(model, data_train, target_train, cv=5)
        mean_score = scores.mean()

        print(f"lr={lr}, max_leaf_nodes={mln} -> CV acc={mean_score:.3f}")

        if mean_score > best_score:
            best_score = mean_score
            best_params = {"learning_rate": lr, "max_leaf_nodes": mln}

print("\nBest CV score:", round(best_score, 3))
print("Best params:", best_params)


lr=0.01, max_leaf_nodes=3 -> CV acc=0.790
lr=0.01, max_leaf_nodes=10 -> CV acc=0.814
lr=0.01, max_leaf_nodes=30 -> CV acc=0.842
lr=0.1, max_leaf_nodes=3 -> CV acc=0.849
lr=0.1, max_leaf_nodes=10 -> CV acc=0.863
lr=0.1, max_leaf_nodes=30 -> CV acc=0.861
lr=1, max_leaf_nodes=3 -> CV acc=0.854
lr=1, max_leaf_nodes=10 -> CV acc=0.837
lr=1, max_leaf_nodes=30 -> CV acc=0.824
lr=10, max_leaf_nodes=3 -> CV acc=0.288
lr=10, max_leaf_nodes=10 -> CV acc=0.646
lr=10, max_leaf_nodes=30 -> CV acc=0.534

Best CV score: 0.863
Best params: {'learning_rate': 0.1, 'max_leaf_nodes': 10}


In [7]:
best_lr = best_params["learning_rate"]
best_mln = best_params["max_leaf_nodes"]

model.set_params(
    classifier__learning_rate=best_lr, classifier__max_leaf_nodes=best_mln
)
model.fit(data_train, target_train)
test_score = model.score(data_test, target_test)

print(f"Test score is  {test_score:.3f}")

Test score is  0.870
